In [14]:
import os

In [15]:
import re

In [16]:
# define the name of the file and folder to be used
filename = 'apillarb.js'

In [17]:
# open and read the content of apillarb.js
with open(filename, 'r') as f:
    data = f.readlines()

In [18]:
# debug
# lets see if this actually opened the correct file
print(data[:10])

["import React, { useEffect, useState, useMemo } from 'react';\n", "import { userSelector, useSelector, useStore } from 'react-redux';\n", "import { useRouter } from 'next/dist/client/router';\n", '\n', "import axios from 'axios';\n", "import { allAutoParts } from '../../actions/autoparts';\n", "import { listApillar } from '../../actions/autoparts';\n", "import { autoPartCount } from '../../actions/autoparts';\n", '\n', "import Head from 'next/head';\n"]


In [19]:
# declare empty list to store auto part names
auto_parts = []

In [20]:
# read auto part names from part-names.js file
with open('part-names.js') as f:
    auto_part_data = f.read()

In [21]:
    # find all auto part names using regular expression
    # auto_part_matches = re.findall(r"partname: '(.+?)'", auto_part_data)

    # find all auto part names using regular expression
    auto_part_matches = re.findall(r"partname:(?:\s*)'(.+?)'", auto_part_data)

In [22]:
# debug feature
print(auto_part_matches)

['Air Box or Air Cleaner', 'Air Condition Vents or Heater Vents', 'Air Flow Meter', 'Air Pump', 'Air Ride Compressor', 'Air Shutter', 'Air Tube/Resonator', 'Alternator', 'Amplifier/Radio', 'Antenna', 'Anti Lock Brake Computer', 'Anti Lock Brake Pump', 'Armrest', 'Ash Tray/Lighter', 'Audiovisual or TV Screen)', 'Automatic Headlight Dimmer', 'Auto Glass', 'Auto. Trans. Cooler', 'Axle Actuator 4WD', 'Axle Assy Fr (4WD w. Housing)', 'Axle Assy Rear With Housing', 'Axle Beam Front 2WD Include I Beam Suspension', 'Axle Beam Rear FWD', 'Axle Flange', 'Axle Housing Only', 'Axle Shaft', 'Back Door Above Rear Bumper)', 'Back Door Glass', 'Back Door Handle, Inside', 'Back Door Handle, Outside', 'Back Door Hinge', 'Back Door Moulding', 'Back Door Shell', 'Back Door Trim Panel', 'Back Glass', 'Back Glass Regulator', 'Back Glass Shock', 'Backing Plate, Front', 'Backing Plate, Rear', 'Backup Camera', 'Backup Light', 'Battery', 'Battery Tray', 'Bed, Pickup', 'Bed Floor Pickup', 'Bed Front Panel Pickup

In [23]:
    # re.findall() to obtain all matches and then filtered out any empty matches using a list comprehension. 
    # This ensures that the auto part names are preserved in the order they appear in the part-names.js file.
    auto_parts = [match for match in auto_part_matches if match]

In [24]:
    print(auto_parts[:10])

['Air Box or Air Cleaner', 'Air Condition Vents or Heater Vents', 'Air Flow Meter', 'Air Pump', 'Air Ride Compressor', 'Air Shutter', 'Air Tube/Resonator', 'Alternator', 'Amplifier/Radio', 'Antenna']


In [25]:
    # reformat auto part names for proper capitalization and list items with hyphens
    auto_parts = []
    
    for part in auto_part_matches:
        if '-' in part:
            # format list item with hypens
            new_part = '-'.join([word.capitalize() for word in part.split('-')])
        else:
            # apply proper capitalization for auto part (capitalization preserved for all occurrences of auto part name)
            new_part = re.sub(r"(?i)\b(A Pillars?|APillars?|A Pillar|APillar)\b", 
                      lambda match: match.group(0).upper() if match.group(0).isupper() else match.group(0).replace(match.group(0), match.group(0).capitalize()), 
                      part)
        auto_parts.append(new_part)

In [26]:
# add custom auto part names to auto_parts list
# commenting out for now, initially I was going to add them manually
# the above function opens up the part-names js file to grap the string by regex pattern

# auto_parts.extend(['apillar', 'engine', 'transmission', 'transfer case', 'alternator', 'ac compressor', 'radiator'])

# reverse the order of the auto parts list
# auto_parts = list(reversed(auto_parts))
auto_parts = auto_parts[::-1]

# loop through each auto part
for auto_part in auto_parts:
    try:
        output_lines = []

        # replace parts of apillarb.js with the auto part name
        for line in data:
            # Replace auto part name in image file paths
            line = re.sub(r"\bA[ -]?Pillars?\b|\ba-pillars?\b|\ba[- ]?pillar(?:s)?\b|\bapillars?\b", 
                          f"{auto_part if 'apillar' not in line.lower() else '-'.join(auto_part.split())}", 
                          line, flags=re.IGNORECASE)
            output_lines.append(line)

        # determine folder name based on auto part name, and convert to lowercase
        folder_name = '-'.join(auto_part.split()).lower()

        # set absolute path of output folder to root directory
        output_folder_path = "/Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/"

        # check if the output folder exists; if not, create it
        if not os.path.exists(output_folder_path):
            os.mkdir(output_folder_path)

        # set absolute path of formatted output folder
        formatted_output_folder_path = os.path.join(output_folder_path, folder_name)

        # check if the formatted output folder exists; if not, create it
        if not os.path.exists(formatted_output_folder_path):
            os.mkdir(formatted_output_folder_path)

        # set filename of output file
        output_filename = 'index.js'

        # set output file absolute path within the formatted output folder
        output_path = os.path.join(formatted_output_folder_path, output_filename)

        # check if the output file exists in the formatted output folder; if not, create it
        if not os.path.exists(output_path):
            with open(output_path, 'w') as f:
                f.writelines(output_lines)

            # print the number of lines in the new file
            print(f"{len(output_lines)} lines written to {os.path.abspath(output_path)}")

        # if the output file exists in the formatted output folder, print a message indicating that it was not created
        else:
            print(f"{output_filename} already exists in the formatted output folder; skipping")
    except Exception as e:
        print(f"An error occurred for auto part: {auto_part}")
        print(e)

        # continue to the next iteration of the loop
        continue
        
    # navigate back to the output_folder_path
    os.chdir(output_folder_path)
    

# move the folder to the current working directory
os.rename(
    os.path.join(output_folder_path, formatted_output_folder_path),
    os.path.join(output_folder_path, filename)
)

23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/wiring-harness-transmission/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/wiring-harness-misc/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/wiring-harness-lamp/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/wiring-harness-engine/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/wiring-harness-dash/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/wiring-harness-body/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/wiring-harness-air-conditioning/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/wiper-motor,-rear/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/wiper-motor-front-windshield

An error occurred for auto part: Trailer Hitch/Tow Hook/Winch
[Errno 2] No such file or directory: '/Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/trailer-hitch/tow-hook/winch'
An error occurred for auto part: Track/Watts Linkage
[Errno 2] No such file or directory: '/Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/track/watts-linkage'
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/torsion-bar/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/torque-convertor/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/tonneau-cover/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/tire/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/timing-gears/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/timing-cover/index.js
An error occurred for auto part: Tim

23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/relay-(misc)/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/rear-window-washer-motor/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/rear-window-defogger/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/rear-suspension-trailing-arm/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/rear-suspension-locating-arm/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/rear-suspension-(see-also-control-arms)/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/rear-seat-belt-assembly/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/rear-lower-valance/index.js
An error occurred for auto part: Rear Knuckle/Stub Axle
[Errno 2] No such file or direct

23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/pitman-arm/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/piston/index.js
An error occurred for auto part: Pickup Cap/Camper Shell
[Errno 2] No such file or directory: '/Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/pickup-cap/camper-shell'
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/pickup-bed-side/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/pickup-bed-front-panel/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/pickup-bed-floor/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/pickup-bed/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/parking-assist-camera/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/park-lamp-rea

23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/hybrid-converter-inverter/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/hub,-lockout-4wd/index.js
An error occurred for auto part: Hub Cap/Wheel Cover Display Without Image
[Errno 2] No such file or directory: '/Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/hub-cap/wheel-cover-display-without-image'
An error occurred for auto part: Hub Cap/Wheel Cover Display With Image
[Errno 2] No such file or directory: '/Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/hub-cap/wheel-cover-display-with-image'
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/hub/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/horn/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/hood-shock/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUt

23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/front-door-trim-panel/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/front-door-switch/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/front-door-shell/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/front-door-regulator/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/front-door-moulding/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/front-door-mirror-glass/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/front-door-mirror/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/front-door-hinge/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/front-door-handle-outside/index.js
23111 lines writ

23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/door-outer-repair-panel-front/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/door-outer-repair-panel-back/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/door-handle,-outside/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/door-handle,-inside/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/door-front/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/door-back-door-above-rear-bumper/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/distributor/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/differential-flange-only/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/differential-case-only/inde

23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/bumper-impact-strip/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/bumper-guard-(rear)/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/bumper-guard-(front)/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/bumper-filler-panel/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/bumper-face-bar-(rear)/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/bumper-face-bar-(front)/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/bumper-energy-absorber-(rear)/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/bumper-energy-absorber-(front)/index.js
23111 lines written to /Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/bumper-end-cap/inde

NotADirectoryError: [Errno 20] Not a directory: '/Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/air-box-or-air-cleaner' -> '/Users/autopartex/Desktop/AutoPartEx/AutoPartExUtilities/apillarb.js'